# Getting all arrivals to all stops of a given line in a given day

## Install dependencies

This notebook requires two dependencies which can be installed with the following command `pip install pandas open-bus-stride-client`.

You can also launch it online at [this URL](https://mybinder.org/v2/gh/hasadna/open-bus-stride-client/HEAD?labpath=notebooks%2Fgetting%20all%20arrivals%20to%20all%20stops%20of%20a%20given%20line%20in%20a%20given%20day.ipynb), when launching online the dependencies are already installed.

## Import dependencies

In [1]:
import datetime

from ipywidgets import DatePicker
from IPython.display import display

# we use pandas to visualize the results we get
import pandas as pd

# The stride client library, used to make the calls to the stride api
import stride

# For local development, set the following to connect to local api:
# stride.config.STRIDE_API_BASE_URL = "http://localhost:8000"

## Pick a date

First, we pick the day we want to analyze, GTFS data queries should be limited to a specific date

You can use the date picker to choose a date which will be used in the next code blocks

In [2]:
date_widget = DatePicker(description='Date:', value=datetime.date(2022,2,3))
display(date_widget)

DatePicker(value=datetime.date(2022, 2, 3), description='Date:')

## Get list of GTFS Routes

For this example we get routes based on route_short_name which is usually the route number in the GTFS data

We look for all routes on the given day matchin route_short_name `360`

In [3]:
gtfs_routes = stride.get('/gtfs_routes/list', {
    'date_from': date_widget.value, 'date_to': date_widget.value,
    'route_short_name': 360,
}, pre_requests_callback='print')
pd.DataFrame(gtfs_routes)

https://open-bus-stride-api.hasadna.org.il/gtfs_routes/list?date_from=2022-02-03&date_to=2022-02-03&route_short_name=360


,id,date,line_ref,operator_ref,route_short_name,route_long_name,route_mkt,route_direction,route_alternative,agency_name,route_type
0,116317,2022-02-03,28774,4,360,היכל המשפט/אבא אבן-ירושלים<->סובת התאנה-אפרת-1#,14360,1,#,אגד תעבורה,3
1,116318,2022-02-03,28775,4,360,סובת התאנה-אפרת<->בנייני האומה/שדרות שז''ר-ירו...,14360,2,#,אגד תעבורה,3
2,116484,2022-02-03,29729,4,360,סובת התאנה-אפרת<->בנייני האומה/שדרות שז''ר-ירו...,14360,2,4,אגד תעבורה,3


## Get list of SIRI rides

Based on this list the GTFS line_refs and operator_refs we can get the list of SIRI rides which occured on these routes

We use the scheduled_start_time field which is populated from the SIRI data to limit the rides to the specific date we want to check

This data contains the duration_minutes field which is the duration of the ride from the SIRI data

In [4]:
siri_rides = stride.get('/siri_rides/list', {
    'scheduled_start_time_from': datetime.datetime.combine(date_widget.value, datetime.time(), datetime.timezone.utc),
    'scheduled_start_time_to': datetime.datetime.combine(date_widget.value, datetime.time(23,59), datetime.timezone.utc),
    'siri_route__line_refs': ','.join([str(gtfs_route['line_ref']) for gtfs_route in gtfs_routes]),
    'siri_route__operator_refs': ','.join([str(gtfs_route['operator_ref']) for gtfs_route in gtfs_routes]),
    'order_by': 'scheduled_start_time asc'
}, pre_requests_callback='print')
pd.DataFrame(siri_rides)

https://open-bus-stride-api.hasadna.org.il/siri_rides/list?scheduled_start_time_from=2022-02-03T00%3A00%3A00.000000%2B0000&scheduled_start_time_to=2022-02-03T23%3A59%3A00.000000%2B0000&siri_route__line_refs=28774%2C28775%2C29729&siri_route__operator_refs=4%2C4%2C4&order_by=scheduled_start_time+asc


,id,siri_route_id,journey_ref,scheduled_start_time,vehicle_ref,updated_first_last_vehicle_locations,first_vehicle_location_id,last_vehicle_location_id,updated_duration_minutes,duration_minutes,journey_gtfs_ride_id,route_gtfs_ride_id,gtfs_ride_id
0,7328882,2149,2022-02-03-57859459,2022-02-03T03:30:00+00:00,7666469,2022-02-03 04:00:19.100013+00:00,386317248,386361039,2022-02-03 10:00:40.620443+00:00,25,None,1775107,1775107
1,7330473,2149,2022-02-03-57859460,2022-02-03T04:00:00+00:00,7667769,2022-02-03 05:01:14.171213+00:00,386356206,386439844,2022-02-03 11:00:52.561954+00:00,28,None,1775096,1775096
2,7330271,2152,2022-02-03-57859311,2022-02-03T04:00:00+00:00,7668269,2022-02-03 05:01:13.466420+00:00,386356207,386466682,2022-02-03 11:00:52.233655+00:00,37,None,1775073,1775073
3,7333515,2149,2022-02-03-57859461,2022-02-03T04:30:00+00:00,7668569,2022-02-03 06:01:47.181321+00:00,386454900,386595343,2022-02-03 11:01:55.402914+00:00,33,None,1775097,1775097
4,7336693,2152,2022-02-03-57859312,2022-02-03T05:00:00+00:00,79180201,2022-02-03 06:01:42.911283+00:00,386570495,386806173,2022-02-03 12:02:24.647821+00:00,44,None,1775085,1775085
5,7336842,2149,2022-02-03-57859462,2022-02-03T05:00:00+00:00,7667169,2022-02-03 06:01:43.658995+00:00,386575435,386800307,2022-02-03 12:02:25.256276+00:00,43,None,1775098,1775098
6,7339413,2149,2022-02-03-57859463,2022-02-03T05:20:00+00:00,7668069,2022-02-03 07:01:40.317626+00:00,386695750,386938085,2022-02-03 12:01:23.210686+00:00,41,None,1775108,1775108
7,7341842,2149,2022-02-03-57859464,2022-02-03T05:40:00+00:00,7666269,2022-02-03 07:00:13.845252+00:00,386806171,387017335,2022-02-03 13:00:50.831377+00:00,36,None,1775109,1775109
8,7343744,2149,2022-02-03-57859465,2022-02-03T06:00:00+00:00,79180201,2022-02-03 07:02:02.710756+00:00,386900935,387192639,2022-02-03 13:03:02.201506+00:00,50,None,1775099,1775099
9,7343746,2152,2022-02-03-57859313,2022-02-03T06:00:00+00:00,7667169,2022-02-03 07:02:02.719829+00:00,386907098,387135054,2022-02-03 13:03:02.207175+00:00,40,None,1775086,1775086


## Choose a ride

There are usually a lot of rides for a given route, the following code gets the first ride which occured at or after 7:00 in the morning

(the rides are already ordered in ascending order based on dates, so first one which is after or on 7 will be the right one)

In [5]:
for siri_ride in siri_rides:
    if int(siri_ride['scheduled_start_time'].split('T')[1].split(':')[0]) >= 7:
        break
siri_ride

{'id': 7350179,
 'siri_route_id': 2152,
 'journey_ref': '2022-02-03-57859314',
 'scheduled_start_time': '2022-02-03T07:00:00+00:00',
 'vehicle_ref': '7668069',
 'updated_first_last_vehicle_locations': datetime.datetime(2022, 2, 3, 8, 1, 56, 323122, tzinfo=datetime.timezone.utc),
 'first_vehicle_location_id': 387254642,
 'last_vehicle_location_id': 387478253,
 'updated_duration_minutes': datetime.datetime(2022, 2, 3, 14, 4, 10, 974741, tzinfo=datetime.timezone.utc),
 'duration_minutes': 42,
 'journey_gtfs_ride_id': None,
 'route_gtfs_ride_id': 1775087,
 'gtfs_ride_id': 1775087}

## Get the ride-stops for this ride

Siri ride-stops contain all the list of stops on this ride, the vehicle locations are then related to the ride stops

this contains data from related tables:

* `gtfs_stop__city` / `gtfs_stop__name`: the stop city/name from the gtfs data
* `gtfs_ride_stop__departure_time`: the planned departure time based on the gtfs data
* `nearest_siri_vehicle_location__recorded_at_time`: the date/time from SIRI data of the relevant bus which was on this route/ride and nearest to this gtfs stop (based on lat/lon)

In [7]:
siri_ride_stops = stride.get('/siri_ride_stops/list', {
    'siri_ride_ids': str(siri_ride['id']),
    'order_by': 'order asc',
    'expand_related_data': True
}, pre_requests_callback='print')
df = pd.DataFrame(siri_ride_stops)
df.loc[:, [
    'order', 'gtfs_stop__city', 'gtfs_stop__name', 'gtfs_ride_stop__departure_time', 
    'nearest_siri_vehicle_location__recorded_at_time'
]]

https://open-bus-stride-api.hasadna.org.il/siri_ride_stops/list?siri_ride_ids=7350179&order_by=order+asc&expand_related_data=True


,order,gtfs_stop__city,gtfs_stop__name,gtfs_ride_stop__departure_time,nearest_siri_vehicle_location__recorded_at_time
0,1,ירושלים,היכל המשפט/אבא אבן,2022-02-03T07:00:00+00:00,2022-02-03T07:00:18+00:00
1,2,ירושלים,בנייני האומה,2022-02-03T07:01:59+00:00,2022-02-03T07:04:19+00:00
2,3,ירושלים,גשר המיתרים/שד' הרצל,2022-02-03T07:03:09+00:00,2022-02-03T07:07:07+00:00
3,4,ירושלים,אצטדיון טדי/א''ס ביתר,2022-02-03T07:08:59+00:00,2022-02-03T07:14:21+00:00
4,5,גוש עציון,מחסום המנהרות,2022-02-03T07:15:00+00:00,2022-02-03T07:21:55+00:00
5,6,אפרת,אפרת שער צפוני/כניסה,2022-02-03T07:19:59+00:00,2022-02-03T07:29:19+00:00
6,7,אפרת,דוד המלך/הדבש,2022-02-03T07:20:59+00:00,2022-02-03T07:30:20+00:00
7,9,אפרת,שדרות רחל אמנו/שדרות דוד המלך,2022-02-03T07:25:59+00:00,2022-02-03T07:33:19+00:00
8,10,אפרת,הדקל/עזרא,2022-02-03T07:28:59+00:00,2022-02-03T07:35:06+00:00
9,11,אפרת,שדרות דוד המלך/השיירות,2022-02-03T07:30:00+00:00,2022-02-03T07:36:14+00:00
